# Task-2
Нужно сгенерировать файл, содержащий 2000 48-битных случайных целых чисел, каждое число на отдельной строке.

Посчитать, какое суммарное количество простых множителей присутствует при факторизации всех чисел. Например, пусть всего два числа: 6 и 8. 6 = 2 * 3, 8 = 2 * 2 * 2. Ответ 5.

Реализовать подсчет

- простым последовательным алгоритмом
- многопоточно, с использованием примитивов синхронизации
- с помощью Akka (или аналога)
- c помощью RxJava (или аналога)

Измерить время выполнения для каждого случая. Использовать уровень параллельности в соответствии с числом ядер вашего CPU.

In [1]:
filename = 'local_numbers.txt'
count = 2000

## Генерация данных
В файл записывпается массив беззнаковых чисел. Навазние файла и его количество чител задается как аргуемнты функции. 

In [2]:
from data_generation import generate_file

In [3]:
generate_file(count, filename)

## Посдеовательная факторизация

Для подсчета делителей используется функция factorint

In [4]:
import numpy as np
from data_generation import read_file
from sympy.ntheory import factorint
from toolz import compose

In [5]:
def count_factors(x: int) -> int:
    return sum(factorint(x).values())

In [6]:
%%timeit 10
sum(np.vectorize(count_factors)(read_file(filename)))

1.65 s ± 72.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Многопоточная факторизация

In [7]:
from multiprocessing.dummy import Pool as thread_pool

In [8]:
%%timeit 10
chunked_arrays = np.array_split(read_file(filename), 8)
sum(sum(thread_pool(8).map(np.vectorize(count_factors), chunked_arrays)))

1.79 s ± 72.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Многопроцессорная факторизация

In [9]:
from multiprocessing import Pool as process_pool

In [10]:
%%timeit 10
chunked_arrays = np.array_split(read_file(filename), 8)
sum(sum(process_pool(8).map(np.vectorize(count_factors), chunked_arrays)))

534 ms ± 7.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Pykka факторизация 1
Два актора: один подсчитывает количество делителей у одного числа, второй суммирует результаты.

In [11]:
import pykka

class Counter(pykka.ThreadingActor):
    def __init__(self):
        super().__init__()
        self.count_divs = 0
    
    def count(self, x):
        self.count_divs += np.sum(x)

class Factoriser(pykka.ThreadingActor):
    def __init__(self, counter: Counter):
        super().__init__()
        self.counter = counter

    def factorise(self, x: np.ndarray) -> np.ndarray:
        self.counter.count(np.vectorize(count_factors)(x))

In [12]:
%%timeit 10
chunked_arrays = np.array_split(read_file(filename), 8)
counter_ref = Counter.start().proxy()
factorisers = [Factoriser.start(counter_ref).proxy().factorise(x) for x in chunked_arrays]
pykka.get_all(factorisers)
result = counter_ref.count_divs.get()
_ = pykka.ActorRegistry.stop_all()
result

1.79 s ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Pykka факторизация 1
Один актор который  считает количество делителей у числа

In [13]:
%%timeit 10
class SimpleFactoriser(pykka.ThreadingActor):
    def on_receive(self, x: np.ndarray) -> np.ndarray:
        return np.sum(np.vectorize(count_factors)(x))

chunked_arrays = np.array_split(read_file(filename), 8)
result = sum([SimpleFactoriser.start().ask(x) for x in chunked_arrays])
_ = pykka.ActorRegistry.stop_all()
result

1.58 s ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## RX факторизация

In [14]:
import rx
from rx import operators as ops

In [15]:
%%timeit 10
source = rx.of(*read_file(filename))
composed = source.pipe(
    ops.map(count_factors),
    ops.sum()
)
composed.subscribe(lambda value: print(f"Received {value}"))

Received 8978
Received 8978
Received 8978
Received 8978
Received 8978
Received 8978
Received 8978
Received 8978
1.65 s ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Наилучший результат получен у многопроцессовой реализации 534ms. Худший результат показала многопоточная реализация: 1.79 s, скорре всего связано с накладными расходами на потоки и работой GIL. 

Так же стоит упомянуть, что у rx реализации не были использованы все преимущества, например реализация чтений с помощью генератора и построения пайплайна напрямую из файла. В текущей реализации происходит полное чтение файла и дальнейшая работа с полной выгрузкой.